In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Function to plot cumulative distance
def plot_cumulative_distance(df, prefix=''):
    """
    Plots the cumulative distance against cumulative time for each maid in the dataframe.

    Parameters:
    df (pd.DataFrame): DataFrame containing maid, datetime, cumulative_distance(kms).
    """
    

    # Check and convert the datetime column if necessary
    if not pd.api.types.is_datetime64_any_dtype(df['datetime']):
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Plotting cumulative distance against cumulative time
    plt.figure(figsize=(12, 6))

    for maid, group in df.groupby('maid'):
        plt.plot(group['datetime'], group['distance'], marker='o', linestyle='-', label=maid)

    plt.xlabel('Date')
    plt.ylabel('Distance (Kms)')

    # Adjusting the legend position and appearance
    plt.legend(
        loc='upper left',
        title='Maid ID',
        fancybox=True,
        framealpha=0.5,
        borderpad=1
    )

    plt.title(f'{prefix} Distance vs Time')

    # Set the x-axis major locator to day and format the dates
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
    # ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))


    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot the cumulative distance vs time using the sample data
# plot_cumulative_distance(df)



In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime

def create_position_time(df, shapefile_path, save_path):
    # Load shapefile using GeoPandas
    gdf = gpd.read_file(shapefile_path)

    # Convert datetime to pandas datetime
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Calculate time duration from a specific date (e.g., 2022-09-03)
    start_date = datetime(2022, 9, 3)
    df['time_duration'] = (df['datetime'] - start_date).dt.total_seconds() / 3600  # in hours
    max_duration = df['time_duration'].max()

    # Normalize time_duration to [0, 1] for heatmap intensity
    df['heatmap_intensity'] = df['time_duration'] / max_duration

    # Calculate bounding box based on dataframe coordinates
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

    # Calculate center of the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2

    # Create the map centered around the bounding box
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Uncomment if you want to add the GeoJSON layer
    # folium.GeoJson(gdf.to_json(), name="geojson").add_to(m)

    # Fit the map to the bounding box
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

    # Define VIBGYOR colors for markers
    vibgyor_colors = ['violet', 'indigo', 'blue', 'green', 'yellow', 'orange', 'red']

    # Define gradient colors for lines
    gradient_colors = ['green', 'lightgreen', 'yellow', 'orange', 'red', 'maroon']

    def get_gradient_color(speed):
        if speed > 600:
            return 'maroon'
        elif speed > 500:
            return 'red'
        elif speed > 400:
            return 'orange'
        elif speed > 300:
            return 'yellow'
        elif speed > 200:
            return 'lightgreen'
        else:
            return 'green'

    # Add lines connecting consecutive points with gradient colors
    for i in range(len(df) - 1):
        point1 = (df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        point2 = (df.iloc[i + 1]['latitude'], df.iloc[i + 1]['longitude'])
        
        # Determine color index based on heatmap intensity for markers
        intensity_index = int(df.iloc[i + 1]['heatmap_intensity'] * (len(vibgyor_colors) - 1))
        marker_color = vibgyor_colors[intensity_index]

        # Determine color for lines based on instantaneous speed
        speed = df.iloc[i + 1]['Velocity']
        line_color = get_gradient_color(speed)
        # Add markers with tooltips for each point
        tooltip1 = f"Maid: {df.iloc[i]['maid']}<br>Lat: {df.iloc[i]['latitude']}<br>Lon: {df.iloc[i]['longitude']}<br>Date: {df.iloc[i]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Displacement: {df.iloc[i]['displacement']}<br>Total Distance: {df.iloc[i]['distance']}<br>Inst. Speed(km/h): {df.iloc[i]['Velocity']}"
        folium.CircleMarker(
            location=point1,
            radius=5,
            color=marker_color,
            fill=True,
            fill_color=marker_color,
            fill_opacity=1,
            tooltip=tooltip1
        ).add_to(m)
        
        tooltip2 = f"Maid: {df.iloc[i+1]['maid']}<br>Lat: {df.iloc[i+1]['latitude']}<br>Lon: {df.iloc[i+1]['longitude']}<br>Date: {df.iloc[i+1]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Displacement: {df.iloc[i+1]['displacement']}<br>Total Distance: {df.iloc[i+1]['distance']}<br>Inst. Speed(km/h): {df.iloc[i+1]['Velocity']}"
        folium.CircleMarker(
            location=point2,
            radius=5,
            color=marker_color,
            fill=True,
            fill_color=marker_color,
            fill_opacity=1,
            tooltip=tooltip2
        ).add_to(m)

        # Add polyline connecting consecutive points with gradient color
        folium.PolyLine(locations=[point1, point2], color=line_color, weight=2.5, opacity=1).add_to(m)
    # Add starting and ending points with star markers
    start_point = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
    end_point = (df.iloc[-1]['latitude'], df.iloc[-1]['longitude'])
    folium.Marker(
        location=start_point,
        icon=folium.Icon(color='silver', icon='star'),
        tooltip='Start Point'
    ).add_to(m)
    folium.Marker(
        location=end_point,
        icon=folium.Icon(color='black', icon='star'),
        tooltip='End Point'
    ).add_to(m)
    # Save map to an HTML file
    m.save(save_path)

# Actual usage
# shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
# data_02a2 = data_dist[data_dist['maid']=='02a2c301-fb8c-4657-a346-a60fd50754d5']
# save_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\position_time_02a2c301-fb8c-4657-a346-a60fd50754d5.html'
# create_position_time(data_02a2, shapefile_path,save_path )


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

def calculate_distances_and_time_gaps_continous_invalid_stamps(data, displacement_threshold, velocity_threshold, time_window, distance_range):
    # Convert 'datetime' column to datetime format
    data['datetime'] = pd.to_datetime(data['datetime'])
    # col_name = f'keep_for_{displacement_threshold}_{velocity_threshold}_{time_window}_{distance_range}'
    # Initialize new columns
    data['dist_previous_next'] = None
    data['time_gap_previous_next'] = None
    data['next_valid_index'] = None
    data['keep'] = True
    
    # Sort data by 'maid', 'datetime', 'latitude', and 'longitude', and reset index
    data = data.sort_values(by=['maid', 'datetime', 'latitude', 'longitude']).reset_index(drop=True)
    
    # Maintain a mapping from the original index to the new sorted index
    original_to_sorted_index = data.reset_index().set_index('index').index
    # Initialize the progress bar
    tqdm.pandas(desc="Processing maids")

    # Calculate distance and time gap
    for maid, group in tqdm(data.groupby('maid'), desc="Processing each maid"):
    # Calculate distance and time gap
    # for maid, group in data.groupby('maid'):
        num_records = len(group)
        for i in range(1, num_records - 1):
        # for i in range(1, num_records - 1):
            displacement = group.loc[group.index[i], 'displacement']
            # print(f'group.index[i]: {group.index[i]}')
            # print(f'disalcement:{displacement}')
            velocity = group.loc[group.index[i], 'Velocity']
            # print(f'disalcement:{displacement}, velocity { {velocity}}')
            if (displacement >= displacement_threshold) and (velocity >= velocity_threshold):
                # print(f'Inside if: disalcement:{displacement}, velocity { {velocity}}')
                # print(f'index: {group.index[i]}')
                prev_record = group.loc[group.index[i - 1]]
                next_valid_index = None
                
                for j in range(i + 1, num_records):
                    next_record = group.loc[group.index[j]]
                    time_gap = (next_record['datetime'] - prev_record['datetime']).total_seconds() / 60
                    
                    if time_gap > time_window:
                        break
                    
                    distance = geodesic(
                        (prev_record['latitude'], prev_record['longitude']),
                        (next_record['latitude'], next_record['longitude'])
                    ).kilometers
                    
                    if distance < distance_range:
                        next_valid_index = j
                        break

                if next_valid_index is not None:
                    original_index = group.index[i]
                    sorted_next_valid_index = group.index[next_valid_index]

                    # Convert sorted index back to original index
                    original_next_valid_index = original_to_sorted_index.get_loc(sorted_next_valid_index)

                    # Update the current record (i) with information about the next valid index
                    data.at[original_index, 'next_valid_index'] = original_next_valid_index
                    data.at[original_index, 'dist_previous_next'] = distance
                    data.at[original_index, 'time_gap_previous_next'] = time_gap

                    # Mark records between the current index and one record before the next valid index as False
                    if original_next_valid_index > original_index:
                        indices_to_mark = range(original_index , original_next_valid_index)
                        data.loc[indices_to_mark, 'keep'] = False

    return data



In [ ]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

def calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward(data, displacement_threshold, velocity_threshold, time_window, distance_range):
    # Convert 'datetime' column to datetime format
    data['datetime'] = pd.to_datetime(data['datetime'])
    # col_name = f'keep_for_{displacement_threshold}_{velocity_threshold}_{time_window}_{distance_range}'
    # Initialize new columns
    data['dist_previous_next'] = None
    data['time_gap_previous_next'] = None
    data['prev_valid_index'] = None
    data['keep'] = True
    
    # Sort data by 'maid', 'datetime', 'latitude', and 'longitude', and reset index
    data = data.sort_values(by=['maid', 'datetime', 'latitude', 'longitude']).reset_index(drop=True)
    
    # Maintain a mapping from the original index to the new sorted index
    original_to_sorted_index = data.reset_index().set_index('index').index
    # Initialize the progress bar
    tqdm.pandas(desc="Processing maids")

    # Calculate distance and time gap
    for maid, group in tqdm(data.groupby('maid'), desc="Processing each maid"):
    # Calculate distance and time gap
    # for maid, group in data.groupby('maid'):
        num_records = len(group)
        for i in range(1, num_records - 1):
        # for i in range(1, num_records - 1):
            displacement = group.loc[group.index[i], 'displacement']
            # print(f'group.index[i]: {group.index[i]}')
            # print(f'disalcement:{displacement}')
            velocity = group.loc[group.index[i], 'Velocity']
            # print(f'disalcement:{displacement}, velocity { {velocity}}')
            if (displacement >= displacement_threshold) and (velocity >= velocity_threshold):
                # print(f'Inside if: disalcement:{displacement}, velocity { {velocity}}')
                # print(f'index: {group.index[i]}')
                curr_record = group.loc[group.index[i]]
                prev_valid_index = None
                
                for j in range(i - 1, 0,-1):
                    prev_record = group.loc[group.index[j]]
                    # print(f'prev_record= {prev_record}')
                    # print(f'index of prev_record: {j}')
                    time_gap = (curr_record['datetime'] - prev_record['datetime']).total_seconds() / 60
                    # print(f'time_gap:{time_gap} {type(time_gap)}')
                    # print(f'time_window: {time_window} {type(time_window)}')
                    if time_gap > time_window:
                        
                        # print('break due to time gap')
                        break
                    
                    distance = geodesic(
                        (prev_record['latitude'], prev_record['longitude']),
                        (curr_record['latitude'], curr_record['longitude'])
                    ).kilometers
                    
                    if distance < distance_range:
                        prev_valid_index = j
                        # print('break due to distance')
                        break

                if prev_valid_index is not None:
                    original_index = group.index[i]
                    sorted_previous_valid_index = group.index[prev_valid_index]

                    # Convert sorted index back to original index
                    original_previous_valid_index = original_to_sorted_index.get_loc(sorted_previous_valid_index)
                    # print(f'original_index: {original_index}')
                    # print(f'original_previous_valid_index: {original_previous_valid_index}')
                    # Update the current record (i) with information about the next valid index
                    data.at[original_index, 'prev_valid_index'] = original_previous_valid_index
                    data.at[original_index, 'dist_previous_next'] = distance
                    data.at[original_index, 'time_gap_previous_next'] = time_gap

                    # Mark records between the one record previous to current index and one record later than the previous valid index as False
                    if original_previous_valid_index < original_index:
                        indices_to_mark = range(original_previous_valid_index + 1, original_index )
                        # print(f'indices_to_mark: {indices_to_mark}')
                        data.loc[indices_to_mark, 'keep'] = False

    return data


In [ ]:
def filter_out_continuos_invalid_stamps(data):
    filtered_data = data[data['keep'] == True]
    return filtered_data

In [ ]:
import pandas as pd
import numpy as np

def optimized_function_maid_vectorized(df, lat_col='latitude', lon_col='longitude', datetime_col='datetime', maid_col='maid'):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(by=[maid_col, datetime_col, lat_col, lon_col])

    # Shift columns to get previous values for each maid
    df['prev_X'] = df.groupby(maid_col)[lon_col].shift(1)
    df['prev_Y'] = df.groupby(maid_col)[lat_col].shift(1)
    df['prev_datetime'] = df.groupby(maid_col)[datetime_col].shift(1)

    # Radius of the Earth in kilometers
    R = 6371

    # Convert latitude and longitude from degrees to radians
    lat1 = np.radians(df[lat_col].values)
    lat2 = np.radians(df['prev_Y'].values)
    dlat = lat2 - lat1
    dlon = np.radians(df['prev_X'].values) - np.radians(df[lon_col].values)

    # Haversine formula for distance calculation
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate consecutive distance in kilometers
    df['displacement'] = R * c
    df['distance'] = df.groupby(maid_col)['displacement'].cumsum()

    # Calculate consecutive time gap in minutes
    df['consecutive_time_gap(minutes)'] = (df[datetime_col] - df['prev_datetime']).dt.total_seconds() / 60

    # Calculate velocity
    df['Velocity'] = df['displacement'] / (df['consecutive_time_gap(minutes)'] / 60)

    # Drop temporary columns
    df = df.drop(columns=['prev_X', 'prev_Y', 'prev_datetime', 'consecutive_time_gap(minutes)'])

    # Fill NaN values with 0
    df = df.fillna(0)

    return df


In [ ]:
def params(step):

    # determine parameters:
    if step==1:
        displacement_min_threshold = 75   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 50      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 60 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==2:
        displacement_min_threshold = 150   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 100      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 120 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==3:
        displacement_min_threshold = 225   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 150      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 180 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step ==4:
        displacement_min_threshold = 300   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 200      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 240 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==5:
        displacement_min_threshold = 375   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 250      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 300 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==6:
        displacement_min_threshold = 300   # distance covered during invalid transition
        min_velocity_threshold = 150       # minimum speed during the invalid transition
        dist_prev_next_threshold = 300      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 360 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==7:
        displacement_min_threshold = 700   # distance covered during invalid transition
        min_velocity_threshold = 500      # minimum speed during the invalid transition
        dist_prev_next_threshold = 350      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 420 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==8:
        displacement_min_threshold = 800   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 400      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 480 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==9:
        displacement_min_threshold = 900   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 450      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 540 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==10:
        displacement_min_threshold = 1000   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 500      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 600 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==11:
        displacement_min_threshold = 1000   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 550      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 660 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==12:
        displacement_min_threshold = 1000   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 600      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 720 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==13:
        displacement_min_threshold = 1000   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 650      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 780 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    elif step==14:
        displacement_min_threshold = 1000   # distance covered during invalid transition
        min_velocity_threshold = 500       # minimum speed during the invalid transition
        dist_prev_next_threshold = 700      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 840 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    if step==15:
        displacement_min_threshold = 50   # distance covered during invalid transition
        min_velocity_threshold = 100       # minimum speed during the invalid transition
        dist_prev_next_threshold = 25      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 30 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    if step==16:
        displacement_min_threshold = 50   # distance covered during invalid transition
        min_velocity_threshold = 100       # minimum speed during the invalid transition
        dist_prev_next_threshold = 15      # maximum permittted distance between previos and next location for the invalid transition stamp
        time_gap_pre_next_threshold = 15 # Time gap(min) between previuos and next time stamp for the invalid record
        sub_folder_name = f'threshold_dist_{displacement_min_threshold}_vel_{min_velocity_threshold}_dist_prev_next_{dist_prev_next_threshold}_time_gap_{time_gap_pre_next_threshold}_min'
    else:
        pass
    return displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold, sub_folder_name

In [ ]:
# df_all_1=pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\all_visitors.csv')
df_all=pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Filtered_records\df_all_without_dups_sorted.csv')

In [ ]:
df_all.rename(columns={'DEVICE_ID': 'maid', 'EVENT_DATE': 'datetime', 'LATITUDE':'latitude', 'LONGITUDE':'longitude'}, inplace=True)
df_all.columns

In [ ]:

# print(f'df_all_1.shape : {df_all_1.shape}')  #(19096604, 5) # 140180 uique IDs

# Remove duplicates based on maid, datetime, latitude, and longitude, keeping the first occurrence
# df_all = df_all_1.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'], keep='first')


# print(f'df_all.shape : {df_all.shape}')  #(8235248, 5) # 140180 uique IDs
# df_all['datetime'] = pd.to_datetime(df_all['datetime'])
# maid_counts = df_all['maid'].value_counts()
# distinct_ids = df_all['maid'].nunique()
# print(f'maid distinct counts:  {distinct_ids} ') 
# maid_single_records = maid_counts[maid_counts == 1].index
# print(f' number of IDs with single time stamp: {len(maid_single_records)}') #7942


#---------------------for whole data


print(f'df_all.shape : {df_all.shape}')  #(41033296, 5) # 388114  uique IDs
df_all['datetime'] = pd.to_datetime(df_all['datetime'])
maid_counts = df_all['maid'].value_counts()
distinct_ids = df_all['maid'].nunique()
print(f'maid distinct counts:  {distinct_ids} ') 
maid_single_records = maid_counts[maid_counts == 1].index
print(f' number of IDs with single time stamp: {len(maid_single_records)}') #59038

In [ ]:
#-----------------for whole data
# Step 1: Identify IDs with a single record
ids_with_single_record = df_all.groupby('maid').filter(lambda x: len(x) == 1)['maid']
print(len(ids_with_single_record)) #59038
# Step 2: Drop records with these IDs
df_all = df_all[~df_all['maid'].isin(set(ids_with_single_record))]
df_all['maid'].nunique() # 329076
df_all.shape # 40974258

In [ ]:
import gc
del df_all_1
gc.collect()


In [ ]:
df_all = df_all[~df_all['maid'].isin(maid_single_records)]
df_all.shape

In [ ]:
# df_all = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between datetimes\df_all_without_duplicates.csv')
#-----------------------for whole data
df_all.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Filtered_records\df_all_without_dups_and_single_record_id_sorted.csv', index=False)

In [ ]:
df_all.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between datetimes\all records without duplicates\all_records_without_duplicates.csv', index=False)

In [ ]:
distinct_ids = df_filtered['maid'].nunique()
print(f'maid distinct counts:  {distinct_ids} ')  # 132238 

In [ ]:
df_filtered = df_filtered.sort_values(by=['maid','datetime','latitude','longitude'])

In [ ]:
df_filtered = optimized_function_maid_vectorized(df_filtered)

In [ ]:
#----------------for whole data
df_all = optimized_function_maid_vectorized(df_all)
df_all.shape

In [ ]:
df_all.head(10)

In [ ]:
df_filtered.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\df_all_without_duplicates.csv', index=False)

In [ ]:
# df_filtered = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\df_all_without_duplicates.csv')

In [ ]:
# df_filtered = df_filtered[['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance', 'Velocity']]

In [ ]:
# for old whole data
step =1
while step<17:
    print(f'records before filtering invalid stamps : {df_filtered.shape}')
    displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold, sub_folder_name = params(step)
    # data_new.to_excel(fr'{folder_path}\data_{folder_name}.xlsx',index=False)  #save_original_dataframe
  
    # state validity_of_records
    df_filtered = calculate_distances_and_time_gaps_continous_invalid_stamps(df_filtered, displacement_min_threshold , min_velocity_threshold,time_gap_pre_next_threshold, dist_prev_next_threshold  )  #dataframe, displacement(km), velocity(kmph)
    # rec_with_issue.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_{sub_folder_name}.csv',index=False)  #save datframe processed at stage 1

    df_filtered = filter_out_continuos_invalid_stamps(df_filtered)  # dataframe, distance_between previpus and next(km), time(minutes)
    print(f'data_new_ shape after filtering out invalid records: {df_filtered.shape}')

    df_filtered = optimized_function_maid_vectorized(df_filtered)


    # plot_cumulative_distance(data_new, f'Filtered:')
    df_filtered.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_filtered_{sub_folder_name}.csv',index=False)  # save dataframe after filtering out individual records
    step+=1

In [ ]:
df_all.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\with displacemnet, distance, velocity\df_all_without_dups_with_disp_vel.csv', index=False)

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv(r"C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_with_states_districts.csv")

In [ ]:
# df_char_dham = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Char dham\current_working_df.csv')
# df_char_dham['datetime'] = pd.to_datetime(df_char_dham['datetime'])

displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold = 100, 150,75, 90
# data_new.to_excel(fr'{folder_path}\data_{folder_name}.xlsx',index=False)  #save_original_dataframe
print(df.shape)
# state validity_of_records
df = calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward(df, displacement_min_threshold , min_velocity_threshold,time_gap_pre_next_threshold, dist_prev_next_threshold  )  #dataframe, displacement(km), velocity(kmph)
# rec_with_issue.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_{sub_folder_name}.csv',index=False)  #save datframe processed at stage 1

df = filter_out_continuos_invalid_stamps(df)  # dataframe, distance_between previpus and next(km), time(minutes)
print(f'data_new_ shape after filtering out invalid records: {df.shape}')
# count_of_rec_at_each_step[step]=len(df_all)

df = optimized_function_maid_vectorized(df)


In [ ]:
import pandas as pd

def remove_unnecessary_timestamps(df):
    """
    Filters the dataframe to keep only the first and last records where latitude and longitude remain the same
    across consecutive timestamps for each maid.

    Parameters:
    df (pd.DataFrame): Input dataframe containing 'maid', 'datetime', 'latitude', and 'longitude' columns.

    Returns:
    pd.DataFrame: Filtered dataframe with only the first and last records where coordinates remain the same for each maid.
    """
    # Round latitude and longitude to 4 decimal places
    df['latitude'] = df['latitude'].astype(float).round(4)
    df['longitude'] = df['longitude'].astype(float).round(4)
    # Sort by 'maid' and 'datetime' to ensure consecutive timestamps are in order within each maid
    df = df.sort_values(by=['maid', 'datetime','latitude','longitude']).reset_index(drop=True)

    # Identify and keep first and last records where lat and long remain the same within each maid
    df['prev_lat'] = df.groupby('maid')['latitude'].shift(1)
    df['prev_long'] = df.groupby('maid')['longitude'].shift(1)
    df['next_lat'] = df.groupby('maid')['latitude'].shift(-1)
    df['next_long'] = df.groupby('maid')['longitude'].shift(-1)

    # Filter to keep rows where latitude or longitude changes within each maid
    filtered_df = df[(df['latitude'] != df['prev_lat']) |
                     (df['longitude'] != df['prev_long']) |
                     (df['latitude'] != df['next_lat']) |
                     (df['longitude'] != df['next_long'])]
    filtered_df.drop(['prev_lat', 'prev_long', 'next_lat', 'next_long'], axis=1, inplace=True)
    # Drop records for maids with only one record
    # Count occurrences of each maid
    maid_counts = df['maid'].value_counts()
    maid_single_records = maid_counts[maid_counts == 1].index
    filtered_df = filtered_df[~filtered_df['maid'].isin(maid_single_records)]

    return filtered_df

df = remove_unnecessary_timestamps(df)

In [ ]:
df['maid'].nunique()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

# Assuming your DataFrame is named `gdf_with_all_activities`

# Find duplicates based on 'maid' and 'datetime'
# duplicates = gdf_with_all_activities[gdf_with_all_activities.duplicated(subset=['maid', 'datetime'], keep=False)]
duplicates = df[df.duplicated(subset=['maid', 'datetime'], keep=False)]

# Display the duplicate records
# print(duplicates)   #1279697
duplicates_count = duplicates.groupby(['maid', 'datetime']).size().shape[0]
duplicates_count


In [ ]:
df[(df['datetime']=='2022-09-01 08:37:01') &(df['maid']=='000010b7-6bd5-4296-9235-2a824209b773')]

In [ ]:
# Shift displacement values to compare with the next record
duplicates['next_displacement'] = duplicates.groupby('maid')['displacement'].shift(-1)

# Find consecutive records where displacement of the next record is > 0.1 km
consecutive_records = duplicates[(duplicates['next_displacement'] < 0.1) & (duplicates['maid'] == duplicates['maid'].shift(-1)) & (duplicates['datetime'] == duplicates['datetime'].shift(-1))]

# Drop the helper column
# consecutive_records = consecutive_records.drop(columns=['next_displacement'])

# Reset index if needed
consecutive_records = consecutive_records.reset_index(drop=True)

# print(consecutive_records)
consecutive_records

In [ ]:
# Drop common records from df1
df_clean = df[~df.apply(tuple, axis=1).isin(consecutive_records.apply(tuple, axis=1))]

In [ ]:
df_clean.shape

In [ ]:
df_clean = optimized_function_maid_vectorized(df_clean)

In [ ]:
df_clean.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_with_states_districts_new_without_duplicates_and_cleaned.csv', index=False)

In [ ]:
df.columns

In [ ]:
df['state'].value_counts()

In [ ]:
df_char_dham.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Char dham\current_working_df_after_removing_type_1_2_again.csv', index=False)

In [ ]:
#--------------------------- for whole data
count_of_rec_at_each_step = {}
step =1
while step<17:
    print(f'records before filtering invalid stamps : {df_all.shape}')
    displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold, sub_folder_name = params(step)
    # data_new.to_excel(fr'{folder_path}\data_{folder_name}.xlsx',index=False)  #save_original_dataframe
  
    # state validity_of_records
    df_all = calculate_distances_and_time_gaps_continous_invalid_stamps(df_all, displacement_min_threshold , min_velocity_threshold,time_gap_pre_next_threshold, dist_prev_next_threshold  )  #dataframe, displacement(km), velocity(kmph)
    # rec_with_issue.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_{sub_folder_name}.csv',index=False)  #save datframe processed at stage 1

    df_all = filter_out_continuos_invalid_stamps(df_all)  # dataframe, distance_between previpus and next(km), time(minutes)
    print(f'data_new_ shape after filtering out invalid records: {df_all.shape}')
    count_of_rec_at_each_step[step]=len(df_all)

    df_all = optimized_function_maid_vectorized(df_all)


    # plot_cumulative_distance(data_new, f'Filtered:')
    step+=1
df_all.to_csv(fr'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\resolving type 1\final_step_{step}_type_1_{sub_folder_name}.csv',index=False)  # save dataframe after filtering out individual records

In [ ]:
df_filtered =pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 2\12_filtered_threshold_dist_1000_vel_500_dist_prev_next_600_time_gap_720_min.csv')

In [ ]:
import json
with open(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\resolving type 1\type_1_count_of_rec_at_each_step.json', 'w') as f:
    json.dump(count_of_rec_at_each_step, f)

In [ ]:
#--------------------------- for whole data
type_2_count_of_rec_at_each_step = {}
type_2_count_of_rec_at_each_step[0] = len(df_all)
step =1
while step<17:
    print(f'records before filtering invalid stamps : {df_all.shape}')
    displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold, sub_folder_name = params(step)
    # data_new.to_excel(fr'{folder_path}\data_{folder_name}.xlsx',index=False)  #save_original_dataframe
  
    # state validity_of_records
    df_all = calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward(df_all, displacement_min_threshold , min_velocity_threshold,time_gap_pre_next_threshold, dist_prev_next_threshold  )  #dataframe, displacement(km), velocity(kmph)
    # rec_with_issue.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_{sub_folder_name}.csv',index=False)  #save datframe processed at stage 1

    df_all = filter_out_continuos_invalid_stamps(df_all)  # dataframe, distance_between previpus and next(km), time(minutes)
    print(f'data_new_ shape after filtering out invalid records: {df_all.shape}')
    type_2_count_of_rec_at_each_step[step]=len(df_all)

    df_all = optimized_function_maid_vectorized(df_all)


    # plot_cumulative_distance(data_new, f'Filtered:')
    step+=1
df_all.to_csv(fr'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Resolving type 2\final_step_{step}_type_2_{sub_folder_name}.csv',index=False)  # save dataframe after filtering out individual records
print('**********************************************')

In [ ]:
df_all.shape

In [ ]:
# x= df_all[(df_all['Velocity']>=150) & (df_all['displacement']>=150)]
x.shape # 589962 total records
x['maid'].nunique() #146079 unique IDs
df_all[df_all['maid'].isin(set(x['maid'].unique().tolist()))].shape #30359255

In [ ]:
import json
with open(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Resolving type 2\type_2_count_of_rec_at_each_step.json', 'w') as f:
    json.dump(type_2_count_of_rec_at_each_step, f)

In [ ]:
# for whole data
# Iterate with tqdm for progress bar
for step in tqdm(range(13, 15), desc='Processing Steps'):
    print(f'Step: {step}')
    print(f'records before filtering invalid stamps : {df_filtered.shape}')
    displacement_min_threshold, min_velocity_threshold, dist_prev_next_threshold, time_gap_pre_next_threshold, sub_folder_name = params(step)
    # data_new.to_excel(fr'{folder_path}\data_{folder_name}.xlsx',index=False)  #save_original_dataframe
  
    # state validity_of_records
    df_filtered = calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward(df_filtered, displacement_min_threshold , min_velocity_threshold,time_gap_pre_next_threshold, dist_prev_next_threshold  )  #dataframe, displacement(km), velocity(kmph)
    # rec_with_issue.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 1\{step}_{sub_folder_name}.csv',index=False)  #save datframe processed at stage 1

    df_filtered = filter_out_continuos_invalid_stamps(df_filtered)  # dataframe, distance_between previpus and next(km), time(minutes)
    print(f'data_new_ shape after filtering out invalid records: {df_filtered.shape}')

    df_filtered = optimized_function_maid_vectorized(df_filtered)


    # plot_cumulative_distance(data_new, f'Filtered:')
    df_filtered.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\stepwise processed records\Type 2\{step}_filtered_{sub_folder_name}.csv',index=False)  # save dataframe after filtering out individual records
    print('**********************************************')

In [ ]:
random_ids = df_all['maid'].sample(n=10, random_state=7) #43, 14

In [ ]:
sample_orig = df_all[df_all['maid'].isin(set(random_ids))]
sample_processed = df_filtered[df_filtered['maid'].isin(set(random_ids))]

In [ ]:
plot_cumulative_distance(sample_orig, prefix='Original')
plot_cumulative_distance(sample_processed, prefix='processed')

In [ ]:
x= df_filtered[(df_filtered['Velocity']>=150) & (df_filtered['displacement']>=150)]
x['maid'].count()

In [ ]:
maid_counts = x['maid'].value_counts()

# Create a distribution of the count of records for each maid
maid_distribution = maid_counts.value_counts().sort_index()

# Display the distribution
print("Distribution of total points per maid with Velocity >= 150 and displacement >= 150:")
print(maid_distribution)

In [ ]:
x[(x['Velocity']>=150)]['maid'].nunique()

In [ ]:
x= df_filtered[(df_filtered['Velocity']>=150) & (df_filtered['displacement']>=50)]
x['maid'].nunique()

In [ ]:
maid_counts = x['maid'].value_counts()

# Create a distribution of the count of records for each maid
maid_distribution = maid_counts.value_counts().sort_index()

# Display the distribution
print("Distribution of total points per maid with Velocity >= 150 and displacement >= 50:")
print(maid_distribution)

In [ ]:
x= df_filtered[(df_filtered['Velocity']>=100) & (df_filtered['displacement']>=50)]
x['maid'].count()

In [ ]:
maid_counts = x['maid'].value_counts()

# Create a distribution of the count of records for each maid
maid_distribution = maid_counts.value_counts().sort_index()

# Display the distribution
print("Distribution of total points per maid with Velocity >= 100 and displacement >= 50:")
print(maid_distribution)

In [ ]:
maid_counts.sum()

In [ ]:
x= df_filtered[(df_filtered['Velocity']>=150) & (df_filtered['displacement']>=150)]
ids_vel_150_disp_150 = x['maid'].unique().tolist()
ids_vel_150_disp_150.sort()

In [ ]:
data_with_type_3= df_filtered[df_filtered['maid'].isin(set(ids_vel_150_disp_150))]

In [ ]:
data_with_type_3.shape

In [ ]:
data_with_type_3_airports = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\data_with_type_3_airports.csv')

In [ ]:
sample_ids = ids_vel_150_disp_150[0:40]
sample_data = data_with_type_3_airports[data_with_type_3_airports['maid'].isin(set(sample_ids))]


In [ ]:
data_with_type_3.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\data_with_type_3.csv', index=False)

In [ ]:
# sample_data.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\sample_data_type_3.xlsx', index=False)

In [ ]:
sample_data = pd.read_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\sample_data_type_3.xlsx')

In [ ]:
sample_data.shape
sample_data['datetime'] = pd.to_datetime(sample_data['datetime'])

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Define the airport coordinates
airport_location = (0.0, 0.0)  # Replace with actual coordinates of the airport

# Sample data 1
data1 = {
    'ID': ['000010b7-6bd5-4296-9235-2a824209b773'] * 9,
    'Timestamp': [
        '2022-09-03 08:08:03', '2022-09-03 08:08:54', '2022-09-03 08:38:14',
        '2022-09-03 08:38:23', '2022-09-03 08:38:23', '2022-09-03 10:07:56',
        '2022-09-03 10:37:10', '2022-09-03 11:10:40', '2022-09-05 02:23:59'
    ],
    'Latitude': [30.44185, 30.44185, 30.43681, 30.43681, 30.43682, 30.43667, 30.43668, 31.68660, 30.50863],
    'Longitude': [77.61102, 77.61102, 77.58271, 77.58271, 77.58272, 77.58273, 77.58281, 76.52180, 77.54575],
    'Displacement': [0, 0, 2.77129408, 0, 0.00102655, 0.016215735, 0.007721104, 171.8414421, 163.2830988],
    'Distance': [0, 0, 2.77129408, 2.77129408, 2.77232063, 2.788536365, 2.796257469, 174.6376995, 337.9207983],
    'Velocity': [0, 0, 5.668556073, 0, float('inf'), 0.010864814, 0.015847191, 307.7757171, 4.163054665],
    'Flag': ['TRUE'] * 9,
    'Location': [None, None, None, None, None, None, 'SARSAWA', 'KANGRA', None]
}

df1 = pd.DataFrame(data1)
df1['Timestamp'] = pd.to_datetime(df1['Timestamp'])

# Add a new column for distance to the airport
df1['Distance_to_Airport'] = df1.apply(
    lambda row: geodesic((row['Latitude'], row['Longitude']), airport_location).kilometers, axis=1
)

# Sample data 2
data2 = [
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-08 08:33:46",
        "Latitude": 30.2172169,
        "Longitude": 78.6891128,
        "Displacement": 0.000287051,
        "Distance": 1.55866988,
        "Velocity": float('inf'),
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 06:42:32",
        "Latitude": 28.6519,
        "Longitude": 77.2315,
        "Displacement": 224.0937869,
        "Distance": 225.6524567,
        "Velocity": 10.11887756,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 13:34:07",
        "Latitude": 28.6519,
        "Longitude": 77.2315,
        "Displacement": 0,
        "Distance": 225.6524567,
        "Velocity": 0,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 16:54:19",
        "Latitude": 30.23016,
        "Longitude": 78.69253,
        "Displacement": 225.4152358,
        "Distance": 451.0676925,
        "Velocity": 67.55701373,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 16:54:29",
        "Latitude": 30.23015,
        "Longitude": 78.69254,
        "Displacement": 0.001469504,
        "Distance": 451.069162,
        "Velocity": 0.529021602,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 16:56:52",
        "Latitude": 30.23015,
        "Longitude": 78.69254,
        "Displacement": 0,
        "Distance": 451.069162,
        "Velocity": 0,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 16:56:53",
        "Latitude": 30.23015,
        "Longitude": 78.69253,
        "Displacement": 0.000960735,
        "Distance": 451.0701228,
        "Velocity": 3.458646964,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-09 16:56:53",
        "Latitude": 30.23015,
        "Longitude": 78.69254,
        "Displacement": 0.000960735,
        "Distance": 451.0710835,
        "Velocity": float('inf'),
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 01:17:41",
        "Latitude": 30.22991,
        "Longitude": 78.69276,
        "Displacement": 0.034042964,
        "Distance": 451.1051265,
        "Velocity": 0.00407863,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 01:27:13",
        "Latitude": 30.22992,
        "Longitude": 78.69275,
        "Displacement": 0.001469506,
        "Distance": 451.106596,
        "Velocity": 0.009248639,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 03:45:35",
        "Latitude": 30.22991,
        "Longitude": 78.69276,
        "Displacement": 0.001469506,
        "Distance": 451.1080655,
        "Velocity": 0.480929232,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 14:12:51",
        "Latitude": 28.6519,
        "Longitude": 77.2315,
        "Displacement": 225.407684,
        "Distance": 676.5157495,
        "Velocity": 15.27411039,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 15:12:51",
        "Latitude": 28.6519,
        "Longitude": 77.2315,
        "Displacement": 225.407684,
        "Distance": 676.5157495,
        "Velocity": 15.27411039,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 16:12:51",
        "Latitude": 28.6519,
        "Longitude": 77.2315,
        "Displacement": 225.407684,
        "Distance": 676.5157495,
        "Velocity": 15.27411039,
        "Flag": True
    },
    {
        "ID": "00007182-8140-450f-b91d-b4522543e0f6",
        "Timestamp": "2022-09-10 16:30:12",
        "Latitude": 30.223162,
        "Longitude": 78.6935,
        "Displacement": 224.8721899,
        "Distance": 901.3879394,
        "Velocity": 777.6559881,
        "Flag": True
    }
]

df2 = pd.DataFrame(data2)
df2['Timestamp'] = pd.to_datetime(df2['Timestamp'])

# Add a new column for distance to the airport
df2['Distance_to_Airport'] = df2.apply(
    lambda row: geodesic((row['Latitude'], row['Longitude']), airport_location).kilometers, axis=1
)

# Display the updated DataFrames
print(df1)
print(df2)


In [ ]:
df.rename(columns = {'ID':'maid', 'Timestamp':'datetime', 'Latitude':'latitude', 'Longitude':'longitude', 'Displacement': 'displacement', 'Distance':'distance'}, inplace =True)

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import timedelta

# df = pd.concat([df1, df2])


# df.reset_index(inplace=True)
# df.drop(columns=['index'], inplace=True)

print(f' df: {df.columns}')


# Modify the record
df.at[22, 'Distance_to_Airport'] = 1.5
df.at[23, 'Distance_to_Airport'] = 1.0  # Example of changing latitude
# print(f' df: {df}')
# Process the DataFrame
processed_df = process_dataframe_for_fixing_velocity(df)

# Display the processed DataFrame
# print(processed_df)
processed_df
# df


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
from tqdm import tqdm

def process_dataframe_for_fixing_velocity(df, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90):
    """
    Process the DataFrame to find and flag records based on given conditions, and adjust datetimes if needed.

    Args:
    - df (DataFrame): The input DataFrame with data.
    - velocity_threshold (float): The threshold for velocity to consider a record.
    - displacement_threshold (float): The threshold for displacement to consider a record.
    - min_velocity (float): The minimum velocity for the ratio of distance to time difference.
    - max_velocity (float): The maximum velocity for the ratio of distance to time difference.

    Returns:
    - DataFrame: The processed DataFrame with records flagged for keeping or removal.
    """
    # Ensure the datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Add new columns for actions chosen and keep flag
    df['Action chosen'] = 'None'
    df['keep'] = True

    # Calculate geodesic distance
    def calculate_geodesic_distance(lat1, lon1, lat2, lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).kilometers

    # Calculate time difference in hours
    def calculate_time_diff_hours(time1, time2):
        return abs((time1 - time2).total_seconds()) / 3600

    # Function to process each group
    def process_group(group):
        max_distance_transition_while_tracing = 20
        # Reset index for group processing
        group = group.reset_index(drop=True)
        
        # Filter relevant records based on velocity and displacement thresholds
        relevant_indices = group[(group['Velocity'] > velocity_threshold) & (group['displacement'] > displacement_threshold)].index
        count = len(relevant_indices)
        # print(f'Group size: {len(group)}, Relevant records count: {count}')
        
        # Process each relevant record
        for i in range(count):
            index = relevant_indices[i]
            record = group.iloc[index]
            # print(f'\nProcessing record at index {index}: {record.to_dict()}')
            
            # Set initial velocity range
            current_min_velocity = min_velocity
            current_max_velocity = max_velocity
            
            # Check distance from airport
            if index > 0:
                distance_from_airport_i = group.loc[index, 'Distance to airport (km)']
                distance_from_airport_prev = group.loc[index-1, 'Distance to airport (km)']
                # print(f'Distance from airport (current): {distance_from_airport_i}, (previous): {distance_from_airport_prev}')
                
                if distance_from_airport_i < 2 and distance_from_airport_prev < 2:
                    current_min_velocity = 500
                    current_max_velocity = 600
                    max_distance_transition_while_tracing = 2
                    # print(f'Updated velocity range: {current_min_velocity} - {current_max_velocity}')
            
            valid_record_found = False
            # Iterate backwards from index i to find a valid previous record
            for l in range(index - 1, -1, -1):
                if index - l > 1:
                    distance_change = calculate_geodesic_distance(
                        group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                        group.loc[l + 1, 'latitude'], group.loc[l + 1, 'longitude']
                    )
                    # print(f'Distance change between index {l} and {l+1}: {distance_change} km')
                    
                    if distance_change < max_distance_transition_while_tracing:
                        distance = calculate_geodesic_distance(
                            group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_diff_hours = calculate_time_diff_hours(
                            group.loc[index, 'datetime'], group.loc[l, 'datetime']
                        )
                        # print(f'Distance to previous record: {distance} km, Time difference: {time_diff_hours} hours')
                        
                        if time_diff_hours > 0 and distance >= 0:
                            velocity_ratio = distance / time_diff_hours
                            # print(f'Calculated velocity ratio: {velocity_ratio}')
                        elif time_diff_hours == 0 and distance ==0:
                            velocity_ratio = 0
                            # print(f'Calculated velocity ratio: {velocity_ratio}')                            
                        elif time_diff_hours== 0:
                            velocity_ratio = np.inf
                            # 
                        # print(f'Calculated velocity ratio: {velocity_ratio}')     


                        if current_min_velocity < velocity_ratio < current_max_velocity:
                            group.at[l, 'Action chosen'] = 'selected'
                            group.loc[l + 1:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l} selected.')
                            break

                        elif velocity_ratio < current_min_velocity:
                            # Calculate time x for comparison
                            distance_l_l1 = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[l, 'latitude'], group.loc[l, 'longitude']
                            )
                            time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)
                            # print(f'Time x for record at index {l+1}: {time_x}')

                            # Calculate new time for record l
                            distance_l_plus_1_index = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[index, 'latitude'], group.loc[index, 'longitude']
                            )
                            new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)
                            # print(f'New time for record at index {l+1}: {new_time}')

                            if new_time >= time_x:
                                group.at[l+1, 'datetime'] = new_time
                                group.at[l+1, 'Action chosen'] = 'modified'
                                
                                # Mark records from l+2 to index-1 for removal
                                group.loc[l + 2:index-1, 'keep'] = False
                                valid_record_found = True
                                # print(f'Record at index {l+1} modified.')
                                break
                    elif distance_change >= max_distance_transition_while_tracing:
                        # print(f'when distance>=20')
                        distance_l_l1 = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)
                        # print(f'Time x for record at index {l+1}: {time_x}')

                        distance_l_plus_1_index = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[index, 'latitude'], group.loc[index, 'longitude']
                        )
                        new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)
                        # print(f'New time for record at index {l+1}: {new_time}')
                        if new_time >= time_x:
                            group.at[l+1, 'datetime'] = new_time
                            group.at[l+1, 'Action chosen'] = 'modified'
                            
                            # Mark records from l+2 to index-1 for removal
                            group.loc[l + 2:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l+1} modified.')
                            break
                        else:
                            group.at[index, 'Action chosen'] = 'Remained same'
                            break


            if not valid_record_found:
                # If no valid modification was made, mark as 'Remained same'
                group.at[index, 'Action chosen'] = 'Remained same'
                # print(f'Record at index {index} remained the same.')

        return group

    grouped = df.groupby('maid')
    # Process each group with a progress bar
    processed_groups = []
    for _, group in tqdm(grouped, total=len(grouped), desc="Processing Groups"):
        processed_groups.append(process_group(group))

    # Combine the processed groups back into a single DataFrame
    df = pd.concat(processed_groups)

    return df


In [ ]:
data_with_type_3_airports.shape

In [ ]:
# data_with_type_3_airports = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\data_with_type_3_airports.csv')
# data_with_type_3_airports['datetime'] = pd.to_datetime(data_with_type_3_airports['datetime'])
# data_with_type_3_airports = data_with_type_3_airports.sort_values(by=['maid','datetime','latitude','longitude'])

In [ ]:
data_with_type_3_airports_vel_fixed = process_dataframe_for_fixing_velocity(data_with_type_3_airports)
# data_with_type_3_airports_vel_fixed.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\data_with_type_3_airports_vel_fixed_stage_1.csv', index=False)

In [ ]:
data_with_type_3_airports_vel_fixed = data_with_type_3_airports_vel_fixed[data_with_type_3_airports_vel_fixed['keep']==True]
data_with_type_3_airports_vel_fixed = optimized_function_maid_vectorized(data_with_type_3_airports_vel_fixed)
print(f' shape of dataframe after filtering: {data_with_type_3_airports_vel_fixed.shape}')
data_with_type_3_airports_vel_fixed.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\data_with_type_3_airports_vel_fixed_stage_final.csv', index=False)

In [ ]:
x= data_with_type_3_airports_vel_fixed[(data_with_type_3_airports_vel_fixed['displacement']>=150) & (data_with_type_3_airports_vel_fixed['Velocity']>=150)]
x['maid'].nunique()

In [ ]:
sample_data_1 = sample_data[sample_data['maid']=='000be82d-6ba7-46b5-bfdb-5ffe621cec13']

In [ ]:
# pd.set_option('display.max_rows', 100)  # Default is 60
# pd.set_option('display.max_columns', 20)  # Default is 20
sample_data_fixed_velocity_final = process_dataframe_for_fixing_velocity(sample_data)
sample_data_fixed_velocity_final.shape # (3279, 13)
# sample_data_fixed_velocity_final

In [ ]:
sample_data_fixed_velocity_final = sample_data_fixed_velocity_final[sample_data_fixed_velocity_final['keep']==True]
sample_data_fixed_velocity_final.shape  #3176
sample_data_fixed_velocity_final = optimized_function_maid_vectorized(sample_data_fixed_velocity_final)

In [ ]:
sample_data_fixed_velocity_final.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\filtering issue from records with in-between timestamps\sample_data_fixed_velocity_final_stage_3.xlsx', index=False)

In [ ]:
# sample_ids = sample_data['maid'].unique().tolist()
# sample_ids.sort()
start_idx =35
end_idx=start_idx+5
plotting_ids = sample_ids[start_idx:end_idx]
# while end_idx<40:

In [ ]:
#plotting distabce time for sample data:
x= sample_data[sample_data['maid'].isin(set(plotting_ids))]
y= sample_data_fixed_velocity_final[sample_data_fixed_velocity_final['maid'].isin(set(plotting_ids))]

plot_cumulative_distance(x,'with type 3 ' )
plot_cumulative_distance(y, 'without type 3 ')

In [ ]:
# print(x.iloc)
x.iloc[252:262]

In [ ]:
y.iloc[252:262]

In [ ]:
import pandas as pd

# Data
data = [
    {"Size": "Closed", "Airport Name": "Rasgovindpur Airstrip", "Latitude": 21.805, "Longitude": 87.047},
    {"Size": "Small", "Airport Name": "CHORHATA", "Latitude": 24.503, "Longitude": 81.220},
    {"Size": "Small", "Airport Name": "Korba Airport", "Latitude": 22.414, "Longitude": 82.719},
    {"Size": "Medium", "Airport Name": "SALEM", "Latitude": 11.783, "Longitude": 78.066},
    {"Size": "Small", "Airport Name": "HIRAKUD", "Latitude": 21.580, "Longitude": 84.006},
    {"Size": "Medium", "Airport Name": "BARAPANI", "Latitude": 25.704, "Longitude": 91.979},
    {"Size": "Small", "Airport Name": "Shirpur Airport", "Latitude": 21.324, "Longitude": 74.957},
    {"Size": "Medium", "Airport Name": "BAGDOGRA", "Latitude": 26.681, "Longitude": 88.329},
    {"Size": "Medium", "Airport Name": "SHOLAPUR", "Latitude": 17.628, "Longitude": 75.935},
    {"Size": "Medium", "Airport Name": "SRINAGAR", "Latitude": 33.987, "Longitude": 74.774},
    {"Size": "Small", "Airport Name": "SULUR", "Latitude": 11.014, "Longitude": 77.160},
    {"Size": "Small", "Airport Name": "TEJU NEW", "Latitude": 27.941, "Longitude": 96.134},
    {"Size": "Small", "Airport Name": "TANJORE", "Latitude": 10.722, "Longitude": 79.102},
    {"Size": "Medium", "Airport Name": "TIRUCHIRAPPALLI", "Latitude": 10.765, "Longitude": 78.710},
    {"Size": "Small", "Airport Name": "TIRUPATI", "Latitude": 13.632, "Longitude": 79.543},
    {"Size": "Large", "Airport Name": "THIRUVANANTHAPURAM INTL", "Latitude": 8.482, "Longitude": 76.920},
    {"Size": "Medium", "Airport Name": "UDAIPUR", "Latitude": 24.618, "Longitude": 73.896},
    {"Size": "Small", "Airport Name": "BENTAYAN", "Latitude": 23.095, "Longitude": 75.885},
    {"Size": "Small", "Airport Name": "Umaria Air Field", "Latitude": 23.533, "Longitude": 80.808},
    {"Size": "Medium", "Airport Name": "VADODARA", "Latitude": 22.336, "Longitude": 73.226},
    {"Size": "Small", "Airport Name": "Vanasthali Airport", "Latitude": 26.408, "Longitude": 75.870},
    {"Size": "Medium", "Airport Name": "VARANASI", "Latitude": 25.452, "Longitude": 82.859},
    {"Size": "Small", "Airport Name": "Vellore Airport", "Latitude": 12.909, "Longitude": 79.067},
    {"Size": "Small", "Airport Name": "Vijay Nagar Airport", "Latitude": 19.884, "Longitude": 73.841},
    {"Size": "Small", "Airport Name": "VISHAKHAPATNAM", "Latitude": 17.721, "Longitude": 83.224},
    {"Size": "Small", "Airport Name": "Warangal Airport", "Latitude": 17.914, "Longitude": 79.602},
    {"Size": "Small", "Airport Name": "YELAHANKA", "Latitude": 13.136, "Longitude": 77.606}
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


In [ ]:
# airport_data

import pandas as pd

# Step 1: Create an empty DataFrame
# df_airport = pd.DataFrame()

# Step 2: Define the data into df_temp


# df_temp = pd.DataFrame(data)

# Step 3: Concatenate df_temp with df_airport
df_airport = pd.concat([df_airport, df], ignore_index=True)

# Print df_airport to verify
print(len(df_airport))


In [ ]:
df_airport.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\airport_data.csv', index=False)

In [ ]:
df_airport